<a href="https://colab.research.google.com/github/grochot/offline_analysis/blob/main/noise_system_offline_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Montowanie dysku Google

Dysk montujemy, aby móc wczytywać pliki z Dysku Google

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


#Files path

In [ ]:
src_path = '/gdrive/MyDrive/AGH/wisniowski/test_data/' #data folder path

# **Data parser**

Program, który zmienia nagłówki w plikach na takie, które obsługuje program a następnie zapisuje na dysku nowe pliki ze zmienionymi nagłówkami.

In [ ]:


data = {}
all_files = glob.glob(os.path.join(src_path, "*.csv"))
all_files_enum = enumerate(all_files)
for fn in all_files_enum:
  dev = os.path.basename(fn[1]).split("_")[0]
  df = pd.read_csv(fn[1], comment='#', low_memory = False)
  df[indx] = df.index
  data[fn[1]] = df

header_list = ['time (s)', 'V (V)', 'f (Hz)', 'AHac (Oe)', 'Vsense (V)', 'Vbias (V)', 'I (A)', 'Phase', 'R (ohm)', 'dI/dH', "dI", 'dV', 'X field (Oe)', 'Y field (Oe)', 'Z field (Oe)',  'Hset (Oe)', 'G',  'I/Phase', 'I/Ax', 'dG',  "dR", 'dI/dV', 'dV/dI',  'dG/dH', 'NdR', 'NdG', 'NdI', 'HdIS', 'dV/dH', 'SPdI',  'HdIS', 'HdRS', 'HdGS',  "G(t)",  "R(t)",  'index']
dopasowania = {}
print(header_list)
for i in range (0, len(all_files)):
   header =  data[all_files[i]].columns.values
   for head in header:
     if head in header_list:
        continue
     if (head in header_list) or (head in list(dopasowania.keys())):
        data[all_files[i]].rename(columns={head: dopasowania[head]}, inplace=True)
        continue
     else:
        #print(list(dopasowania.values()))
        new_header = input("Enter new header for " + head + " in " + all_files[i] + ": ")
        dopasowania[head] = new_header
        data[all_files[i]].rename(columns={head: new_header}, inplace=True)

   data[all_files[i]].to_csv(all_files[i]+"_headers_ok", index=False)
   print("Created file: " + all_files[i]+"_headers_ok")


NameError: ignored

# Wczytywanie plików

In [ ]:
from collections import defaultdict
import pandas as pd
import glob
import os
import numpy as np
from scipy.optimize import curve_fit
from scipy.fft import fft, rfft
from scipy.fft import fftfreq, rfftfreq
import altair as alt
import ipywidgets as widgets
#from altair_data_server import data_server
alt.data_transformers.disable_max_rows()
import logging

logger = logging.getLogger('LOG:')

logging.basicConfig(level=logging.DEBUG)
logger.setLevel(logging.INFO)



time = 'time (s)'
V = 'V (V)'
f = 'f (Hz)'
AHac = 'AHac (Oe)'
Vsense = 'Vsense (V)'
Vbias = 'Vbias (V)'
I = 'I (A)'
Phase = 'Phase'
R = 'R (ohm)'
dI_dH = 'dI/dH'
dI = "dI"
dV = 'dV'
X_field = 'X field (Oe)'
Y_field = 'Y field (Oe)'
Z_field = 'Z field (Oe)'
H_set = 'Hset (Oe)'
G = 'G'
I_phase = 'I/Phase'
I_Ax = 'I/Ax'
dG = 'dG'
dR = "dR"
dI_dV = 'dI/dV'
dV_dI = 'dV/dI'
dG_dH = 'dG/dH'
NdR = 'NdR'
NdG = 'NdG'
NdI = 'NdI'
HdIS = 'HdIS'
dV_dH = 'dV/dH'
SPdI = 'SPdI'
HdIS = 'HdIS'
HdRS = 'HdRS'
HdGS = 'HdGS'
G_t = "G(t)"
R_t = "R(t)"
indx = 'index'

# Shortcut to the GDrive
src_path = '/gdrive/MyDrive/AGH/wisniowski/test_data/' #data folder path

data = {}
all_files = glob.glob(os.path.join(src_path, "*.csv_headers_ok"))
all_files_enum = enumerate(all_files)
for fn in all_files_enum:
  dev = os.path.basename(fn[1]).split("_")[0]
  df = pd.read_csv(fn[1], comment='#', low_memory = False)
  df[indx] = df.index
  data[fn[1]] = df.dropna()

logger.info("Files loaded:"+str(len(all_files)))


INFO:LOG::Files loaded:21


# Funkcje liczące

In [ ]:
#Functions

def NLI(list_v_i, list_H, *args):
  results = []
  for i in range(1,len(list_v_i)-1):
    result = (list_v_i[i+1]-list_v_i[i])-(list_v_i[i]-list_v_i[i-1])
    results.append(result)
  return list_H[1:len(list_v_i)-1], results

def NLV(list_v_i, list_H, *args):
  results = []
  for i in range(1,len(list_v_i)-1):
    result = (list_v_i[i+1]-list_v_i[i])-(list_v_i[i]-list_v_i[i-1])
    results.append(result)
  return list_H[1:len(list_v_i)-1], results

def HSI(list_i, list_H, *args):
  return list_H, list_i

def diffJ(A, list_I, list_H, *args):
  results = []
  for i in range(1,len(list_I)-1):
    result = (list_I[i+1]/A-list_I[i-1]/A)
    results.append(result)
  return list_H[1:len(list_I)-1], results

def diffHDC(list_H, *args):
  results = []
  for i in range(1,len(list_H)-1):
    result = (list_H[i+1]-list_H[i-1])
    results.append(result)
  return list_H[1:len(list_H)-1],results

def NdiffJ(A, list_I, list_H, *args):
  j_n = []
  j = diffJ(A, list_I, list_H)[1]
  j_max = max(j)
  j_min = min(j)
  for i in j:
    if j_max - j_min != 0:
      j_n.append(i/(j_max-j_min))
    else:
      j_n.append(i/(1e-20))
  return list_H[1:len(list_I)-1], j_n

def I_min_max(list_i_v, list_H, *args):
  try:
    h_min = list_H.index(min(list_H))
    h_max = list_H.index(max(list_H))
    h_0 = list_H.index(0)
    return list_i_v[h_min], list_i_v[h_max], list_i_v[h_0]
  except:
    return list_i_v[h_min], list_i_v[h_max]


def V_min_max(list_i_v, list_H, *args):
  try:
    h_min = list_H.index(min(list_H))
    h_max = list_H.index(max(list_H))
    h_0 = list_H.index(0)
    return list_i_v[h_min], list_i_v[h_max], list_i_v[h_0]
  except:
    return list_i_v[h_min], list_i_v[h_max]


def dFitI(list_i_v, list_h, start, stop, *args):
  coef = np.polyfit(list_h[start:stop],list_i_v[start:stop],1)
  print(coef)
  poly1d_fn = np.poly1d(coef)
  (print(poly1d_fn))
  differences = []
  for i in range(len(list_i_v)):
    differences.append(list_i_v[i]-poly1d_fn(list_h[i]))
  return list_h, differences

def dFitV(list_i_v, list_h, start, stop, *args):
  coef = np.polyfit(list_h[start:stop],list_i_v[start:stop],1)
  poly1d_fn = np.poly1d(coef)
  differences = []
  for i in range(len(list_i_v)):
    differences.append(list_i_v[i]-poly1d_fn(list_h[i]))
  return list_h, differences

def dfiI(list_i, list_H, *args):
  results = []
  i_max = max(list_i)
  i_min = min(list_i)
  for i in range(len(list_i)-1):
    if i_max-i_min !=0:
      results.append(180-np.arccos(2*(list_i[i+1]-list_i[i])/(i_max-i_min))*180/np.pi)
    else:
      results.append(180-np.arccos(2*(list_i[i+1]-list_i[i])/(1e-15))*180/np.pi)
  return list_H[1:len(list_i)-1], results

def HSR(list_i, list_H, *args):
  return list_H, list_i   #TO DO


def HSG(list_i, list_H, *args):
  return list_H, list_i     #TO DO

def dFitR(list_r_g, list_h, start, stop, *args):
  coef = np.polyfit(list_h[start:stop],list_r_g[start:stop],1)
  poly1d_fn = np.poly1d(coef)
  differences = []
  for i in range(len(list_r_g)):
    differences.append(list_r_g[i]-poly1d_fn(list_h[i]))
  return list_h, differences                              #TO DO

def dFitG(list_r_g, list_h, start, stop, *args):
  coef = np.polyfit(list_h[start:stop],list_r_g[start:stop],1)
  poly1d_fn = np.poly1d(coef)
  differences = []
  for i in range(len(list_r_g)):
    differences.append(list_r_g[i]-poly1d_fn(list_h[i]))
  return list_h, differences                                # TO DO


def R_min_max_0(list_r_g, list_H, *args):
  try:
    h_min = list_H.index(min(list_H))
    h_max = list_H.index(max(list_H))
    h_0 = list_H.index(0)
    return list_r_g[h_min], list_r_g[h_max], list_r_g[h_0]
  except:
    return list_r_g[h_min], list_r_g[h_max]

def G_min_max_0(list_r_g, list_H, *args):
  try:
    h_min = list_H.index(min(list_H))
    h_max = list_H.index(max(list_H))
    h_0 = list_H.index(0)
    return list_r_g[h_min], list_r_g[h_max], list_r_g[h_0]
  except:
    return list_r_g[h_min], list_r_g[h_max]

def d2R(list_dR_dG, list_dI_V, list_H, *args):
  results = []
  for i in range(len(list_dI_V)):
    results.append(list_dR_dG[i]/list_dI_V[i])
  return list_H, results

def d2G(list_dR_dG, list_dI_V, list_H, *args):
  results = []
  for i in range(len(list_dI_V)):
    results.append(list_dR_dG[i]/list_dI_V[i])
  return list_H, results


def FitR(xdata, ydata, start, stop, *args):
  def func(x:list, a:float, b:float, c:float):
    y = float(a) + np.multiply(float(b),x) + np.multiply(float(c),x)
    return y
  popt, pcov = curve_fit(func, xdata[start:stop], ydata[start:stop])
  return xdata, func(xdata, *popt), popt


def FitG(xdata, ydata, start, stop, *args):
  def func(x:list, a:float, b:float, c:float):
    y = float(a) + np.multiply(float(b),x) + np.multiply(float(c),x)
    return y
  popt, pcov = curve_fit(func, xdata[start:stop], ydata[start:stop])
  return xdata, func(xdata, *popt), popt


def TdI(list_time, list_I, list_HAC, *args):
  results = []
  for i in range(1,len(list_time)-1):
    dI = list_I[i+1]-list_I[i-1]
    dt = list_time[i+1] - list_time[i-1]
    results.append(dI/dt)
  return list_HAC[1:len(list_time)-1], results


def FTIt(signal, sampling_rate, *args):
    time_step = 1.0/sampling_rate
    duration = len(signal)/sampling_rate
    time_axis = np.arange(0, duration, time_step)
    frequencies = rfftfreq(len(signal), d = time_step)
    fourier = rfft(signal)
    amplitude = 2*np.abs(fourier)/len(signal)
    return frequencies, amplitude


function_list = ['NLI','NLV', 'HSI', 'diffHDC', 'diffJ', 'NdiffJ', 'I_min_max','V_min_max', 'dFitI','dFitV', 'dfiI', 'HSR', 'HSG', 'dFitR', 'dFitG', 'R_min_max_0','G_min_max_0', 'dR/dI', 'dG/dV', 'FitG', 'FitR', 'TdI', 'FTIt']

##Funkcje wyświetlające

In [ ]:
def function_pik(k, *args):
  for chart in file_picker.value:
    match k:
      case 'NLI':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][I].to_list()},{data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0][1:100], I:results[1][1:100]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y=I).mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )

      case 'NLV':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][V].to_list()},{data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0][1:100], V:results[1][1:100]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y=V).mark_point(size=10).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'HSI':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][I].to_list()},{data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0][1:100], I:results[1][1:100]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y=I).mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'diffHDC':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0][1:100], 'diffH_{DC}':results[1][1:100]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y='diffH_{DC}').mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'diffJ':
        try:
          results = eval(f'{w.children[0].value}'+f'({area_field.children[0].value },{data[chart][I].to_list()},{data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0][1:100], I:results[1][1:100]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y=I).mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'NdiffJ':
        try:
          results = eval(f'{w.children[0].value}'+f'({area_field.children[0].value },{data[chart][I].to_list()},{data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0][1:100], I:results[1][1:100]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y=I).mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )

      case 'I_min_max':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][I].to_list()},{data[chart][H_set].to_list()})')
          print(str(chart)+":\n"+"Minimal I(A):" + str(results[0])+f"\n"+"Maximal I(A):"+str(results[1])+"\n")
          return
        except Exception as error:
          print("Chart could not be displayed due to error:", error )

      case 'V_min_max':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][V].to_list()},{data[chart][H_set].to_list()})')
          print(str(chart)+":\n"+"Minimal V(V):" + str(results[0])+f"\n"+"Maximal V(V):"+str(results[1])+"\n")
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'dFitI':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][dI].to_list()},{data[chart][H_set].to_list()},{int(l.children[0].value)}, {int(p.children[0].value)})')
          pf = pd.DataFrame({H_set:results[0], 'dFitI':results[1]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y='dFitI').mark_line(color = "red").properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'dFitV':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][dV].to_list()},{data[chart][H_set].to_list()},{int(l.children[0].value)}, {int(p.children[0].value)})')
          pf = pd.DataFrame({H_set:results[0][1:100], 'dFitV':results[1][1:100]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y='dFitV').mark_line(color = "red").properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'dfiI':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][I].to_list()},{data[chart][H_set].to_list()})')
          pf = pd.DataFrame({H_set:results[0][1:100], I:results[1][1:100]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y=I).mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )

      case 'HSR':  #TO DO
        pass
      case 'HSG':   #TO DO
        pass
      case 'dFitR':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][dR].to_list()},{data[chart][H_set].to_list()},{int(l.children[0].value)}, {int(p.children[0].value)})')
          pf = pd.DataFrame({H_set:results[0][1:100], 'dFitR':results[1][1:100]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y='dFitR').mark_line(color = "red").properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'dFitG':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][dG].to_list()},{data[chart][H_set].to_list()},{int(l.children[0].value)}, {int(p.children[0].value)})')
          pf = pd.DataFrame({H_set:results[0], 'dFitG':results[1]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y='dFitG').mark_line(color = "red").properties(title = chart).interactive() )
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'R_min_max_0':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][R].to_list()},{data[chart][H_set].to_list()})')
          if len(results) ==2:
            print(str(chart)+":\n"+"Minimal R(Ohms):" + str(results[0])+f"\n"+"Maximal R(Ohms):"+str(results[1])+"\n")
          else:
            print(str(chart)+":\n"+"Minimal R(Ohms):" + str(results[0])+f"\n"+"Maximal R(Ohms):"+str(results[1])+"\n"+ "Zero H:" + str(results[2])+"\n")

        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'G_min_max_0':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][G].to_list()},{data[chart][H_set].to_list()})')
          if len(results) ==2:
            print(str(chart)+":\n"+"Minimal G(1/Ohms):" + str(results[0])+f"\n"+"Maximal G(1/Ohms):"+str(results[1])+"\n")
          else:
            print(str(chart)+":\n"+"Minimal G(1/Ohms):" + str(results[0])+f"\n"+"Maximal G(1/Ohms):"+str(results[1])+"\n"+ "Zero H:" + str(results[2])+"\n")
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'dR/dI':
        try:
          results = eval(f'd2R'+f'({data[chart][dR].to_list()},{data[chart][dI].to_list()},{data[chart][H_set].to_list()} )')
          pf = pd.DataFrame({'dRdI':results[1][1:100], H_set:results[0][1:100]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y='dRdI').mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'dG/dV':
        try:
          results = eval(f'd2G'+f'({data[chart][dG].to_list()},{data[chart][dV].to_list()},{data[chart][H_set].to_list()} )')
          pf = pd.DataFrame({'dGdV':results[1][1:100], H_set:results[0][1:100]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=H_set, y='dGdV').mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'FitG':
        try:
          results = FitG(data[chart][H_set].to_list(),data[chart][G].to_list(),int(l.children[0].value), int(p.children[0].value))
          pf = pd.DataFrame({H_set:results[0], 'FitG':results[1]})
          print("g0:"+f"{results[2][0]}"+"\n"+"g1:"+f"{results[2][1]}"+"\n"+"g2:"+f"{results[2][2]}"+"\n")
          display(alt.Chart(data[chart]).encode(x=H_set, y=G).mark_point().interactive()+alt.Chart(pf).encode(x=H_set, y='FitG').mark_line(color = "red").properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'FitR':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][H_set].to_list()},{data[chart][R].to_list()},{int(l.children[0].value)}, {int(p.children[0].value)})')
          pf = pd.DataFrame({H_set:results[0], 'FitR':results[1]})
          print("g0:"+f"{results[2][0]}"+"\n"+"g1:"+f"{results[2][1]}"+"\n"+"g2:"+f"{results[2][2]}"+"\n")
          if args[0] == True:
            display(alt.Chart(data[chart]).encode(x=H_set, y=R).mark_point().interactive() + alt.Chart(pf).encode(x=H_set, y='FitR').mark_line(color = "red").properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'TdI':
        try:
          results = eval(f'{w.children[0].value}'+f'({data[chart][time].to_list()},{data[chart][I].to_list()},{data[chart][AHac].to_list()} )')
          pf = pd.DataFrame({'TdI':results[1][1:100], AHac:results[0][1:100]})
          if args[0] == True:
            display(alt.Chart(pf).encode(x=AHac, y='TdI').mark_point(size=1).properties(title = chart).interactive())
          return pf
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case 'FTIt':
        try:
          sampling_rate = 2
          results1 = eval(f'TdI'+f'({data[chart][time].to_list()},{data[chart][I].to_list()},{data[chart][AHac].to_list()} )')
          pf1 = pd.DataFrame({'TdI':results1[1][1:100], AHac:results1[0][1:100]})

          results2 = eval(f'{w.children[0].value}'+f'({pf1["TdI"]}, {sampling_rate})')
          pf2 = pd.DataFrame({f:results2[0][1:100], 'FTIt':results2[1][1:100]})
          if args[0] == True:
            display(alt.Chart(pf2).encode(x=f, y='FTIt').mark_point(size=1).properties(title = chart).interactive())
          return pf2
        except Exception as error:
          print("Chart could not be displayed due to error:", error )
      case _ :
        pass



# Główny program

In [ ]:

from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout
try:
  file_picker = widgets.SelectMultiple(
        options=list(all_files),
        value= [all_files[0]],
        description='Files',
        enabled = False)
  output = widgets.Output()
  def f(x):
      output.clear_output()
      k = w.children[0].value
      with output:
        function_pik(k, True)
      return x

  w = interactive(f, x = widgets.Dropdown(
      options=function_list,
      description='Function list:',
      disabled=False,
  ))
  l = interactive(f,x = widgets.BoundedIntText(
      value=0,
      step=1,
      max = 100000000,
      description='Start fit point:',
      disabled=False
  ))
  p = interactive(f,x = widgets.BoundedIntText(
      value=0,
      step=1,
      max = 100000000,
      description='Stop fit point:',
      disabled=False
  ))

  area_field = interactive(f,x = widgets.BoundedFloatText(
      value=0.000001,
      min=0,
      description='Area:',
      disabled=False
  ))
except Exception as k:
  print(k)

def main():
  try:


  # button = widgets.Button(description="Generate graph")




    # display(w)
    # display(l)
    # display(k)
    # button.on_click(ff())
    # title = widgets.Label(value="The $m$ in $E=mc^2$:")
    # title.style.font_size = "100px"

    grid = GridspecLayout(6, 1)
    grid[0, 0] = w
    grid[1, 0] = l
    grid[2, 0] = p
    grid[3, 0] = area_field
    grid[4, 0] = file_picker
    # grid[4, 0] = button



  except IndexError:
    print("Twój folder jest pusty!")
  except NameError:
    print('Zanimu uruchomisz program wczytaj dane!')
  return AppLayout(header=None,
              left_sidebar=grid,
              center=output,
              right_sidebar=None,
              footer=None)

#Wizualizacja

In [ ]:
main()

AppLayout(children=(GridspecLayout(children=(interactive(children=(Dropdown(description='Function list:', opti…

# Zapis do pliku

In [ ]:
########## Save Choosen Data ###########

function_pik(w.children[0].value, False).to_csv(os.path.basename(src_path+file_picker.value[0]).split("/")[0][:-15]+"_"+w.children[0].value+".csv", index=False)
